### The Process

In [38]:
# We pick the topics ahead of time even if we're not sure what the topics are
# Each document is represented as a distribution over topics
# Each document is presented as a distribution over topics.
# Each topic is represented as a distribution of words

In [39]:
# Probability Distribution
# Every docuement is a distribution of topics
# Every topic is a distribution of words

In [40]:
# Goal: You want LDA to learn the topic mix in each document, and the word mix in each topic

# Choose the number of topics you think there are in your corpus
# K = 2

# Randomly assign each word in each document to one of two topics

# Go through every word and its topic assignment in each document. Look at (1) how often the topic occurs in the document\
# and (2) how often the word occurs in the topic overall. Based on this info, assign the word a new topic.

# Go thorugh multiple iterations of this. Eventually the topics will start making sense.

In [41]:
# Input: Document-Term Matrix, Number of topics, Number of iterations

# Gensim will go through the process of finding the best word distribution\ 
# for each topic and best topic distribution for each document

### Text Cleaning

In [72]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

# import the necessary libraries for LDA with Gensim
from gensim import matutils, models
import scipy.sparse

In [43]:
# df = pd.read_csv('New_Data.csv', usecols = ['all_features'], low_memory = True)
# df.head()

In [44]:
# Convert columns to string
# pre_text = df.squeeze()
# text = ' '.join(pre_text)
# print(text)

In [45]:
import spacy
spacy.load('en_core_web_sm')
from spacy.lang.en import English
parser = English()

In [46]:
# Clean text and return a list of tokens
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [47]:
# Use NLTK's wordnet to find the meaning of words, synonoms, antonyms, and more.
# Use word lemmatizer to get the root word.

In [48]:
import nltk
nltk.download('wordnet')

from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ahmad\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [49]:
# Filter out stopwords

In [50]:
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ahmad\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [51]:
# Define a fucntion that prepares the text for topic modelling

In [52]:
# def prepare_text_for_lda(text):
#     tokens = tokenize(text)
#     tokens = [token for token in tokens if len(token) > 4]
#     tokens = [token for token in tokens if token not in en_stop]
#     tokens = [get_lemma(token) for token in tokens]
#     return tokens
# def prepare_text_for_lda(text):
#     new_tokens = []
#     for token in tokenize(text):
#         if (len(token) > 4 and token not in en_stop):
#             new_tokens.append(get_lemma(token))
#     return new_tokens

# retuns a list -  list is full of token 
def prepare_text_for_lda(text):
    return [get_lemma(token) for token in tokenize(text) if (len(token) > 4 and token not in en_stop)]

In [53]:
# Open our data read it in line by line. For each line, prepare text for LDA, then add to a list

In [54]:
import random
text_data = []
with open('dataset.csv', encoding='utf-8') as f:
    for line in f:
        tokens = prepare_text_for_lda(line)
        if random.random() > .99:
            print(tokens)
            text_data.append(tokens)

['55,"tedious', 'slowly', 'remove', 'every', 'single', 'enthusiasm', 'environment', 'culture', 'balance', 'feedback', 'take', 'seriously', 'management', 'frustrate', 'stack', 'slowly', 'erode', 'enthusiasm', 'might']
['66,growing', 'company', 'driving', 'towards', 'limitless', 'learning', 'things', 'notice']
['126,"great', 'place', 'flexibility', 'profesionalism', 'compromise', 'think', 'aways', 'depend', 'business']
['164,"great', 'place', 'great', 'place', 'friendly', 'environment', 'better', 'management', 'support', 'logistics', 'support-', 'computer', 'office', 'phone']
['263,worth', 'explore', 'strong', 'vision', 'focus', 'employee', 'safety', 'lack', 'communication', 'expectation']
['304,"great', 'environment', 'people', 'competitive', 'great', 'people', 'benefit', 'really', 'opportunity', 'area', 'interest', 'holiday', 'could', 'better', 'boarding']
['358,"great', 'company', 'environment', 'benefit', 'would', 'recommend', 'work']
['408,"a', 'great', 'place', 'great', 'benefit', 

### LDA with Gensim

In [55]:
# First create a dictionary from the data, then convert to bag-of-words corpus and save the dictionary and corpus for future use.

In [56]:
from gensim import corpora
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]

import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [57]:
# Define the number of topics you want LDA to find

In [81]:
import gensim
NUM_TOPICS = 5
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=5)
ldamodel.save('model5.gensim')

topics = ldamodel.print_topics(num_words=5)
for topic in topics:
    print(topic)

(0, '0.065*"slowly" + 0.065*"enthusiasm" + 0.036*"environment" + 0.035*"erode" + 0.035*"balance"')
(1, '0.014*"great" + 0.014*"place" + 0.014*"benefit" + 0.014*"environment" + 0.014*"company"')
(2, '0.050*"place" + 0.050*"flexibility" + 0.050*"profesionalism" + 0.050*"think" + 0.050*"aways"')
(3, '0.083*"great" + 0.043*"benefit" + 0.043*"communication" + 0.043*"company" + 0.025*"place"')
(4, '0.050*"place" + 0.050*"better" + 0.050*"great" + 0.050*"environment" + 0.050*"people"')


### PYLDAVIS

In [59]:
# Used to interpret the topics in a topic model that has been fit to a corpus of text data
# Package extracts information from a fitted LDA topic model to inform an interactive web-based visualisation.

In [60]:
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model5.gensim')

import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)